<font color='DarkViolet ' style="font-size:25px"><b>Investment with limitations

We will try to establish an investment scheme for the group of investors. Due to the complex governance, there are some limitations imposed. In column A you will find the name of the asset. In column B (see stocks.csv), you will find the price for the single unit of that asset (price of one stock). Every investor wants to invest in only one asset, if possible, if not, two. The maximum of companies one investor can invest in is, therefore, two.

Every investor can invest a maximum $23000 and can hold not more than 33 stocks. The goal for this group of investors is to purchase 33 stocks of every asset. Our goal is to determine the minimum number of investors to purchase all 33 stocks from every company listed on the table.

Example: Since the second stock on our list (AZO) costs 2461.63 our first investor can buy a maximum of 9 stocks of that, and she is left with the $4845.33 that she can either buy two stocks of CTXS or 18 stocks of ACN. That way, our first investor maxed out his capital (there are still some coins left in both cases) and didn’t break the limit of owning more than 33 stocks (of any combination of companies).

Find the optimal solution of this problem - **what is the minimum number of investors** who all have $23000, so they can purchase 33 stocks of all 140 assets without breaking the rule of one investor owning more than 33 stocks of any combination of assets.


**Hint 1:** This problem could be formulated as a binary linear optimization problem. This type of problem is well established from research perspective, and there are algorithms that find a solution in an acceptable time. In Python the optimization part is done with the help of the PuLP library.

**Hint 2:** There is amazing and detailed math/python blog post about binary linear optimization done by a talented guy Piero Paialunga. You can read it here (https://towardsdatascience.com/hands-on-integer-binary-linear-optimization-using-python-b6d8160cb1de) and use everything that you can find useful.

In [1]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 26.5 MB/s eta 0:00:00


In [77]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
import math
from pulp import *

In [ ]:
from google.colab

In [12]:
from google.colab import files
uploaded = files.upload()

Saving Stocks.csv to Stocks.csv


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df = pd.read_csv(io.StringIO(uploaded['Stocks.csv'].decode('utf-8')))
df

,Asset Name,Price
0,"NVR, Inc. (XNYS:NVR)",4564.07
1,"AUTOZONE, INC. (XNYS:AZO)",2461.63
2,"CITRIX SYSTEMS, INC. (XMEX:CTXS*)",2103.70
3,PT Aneka Tambang Tbk (XIDX:ANTM),2010.00
4,BOOKING HOLDINGS INC. (XNAS:BKNG),1958.84
...,...,...
135,"UNITED PARCEL SERVICE, INC. (XNYS:UPS)",173.61
136,FEDEX CORPORATION (XNYS:FDX),173.05
137,CME GROUP INC. (XNAS:CME),167.94
138,"KEYSIGHT TECHNOLOGIES, INC. (XNYS:KEYS)",167.66


<font color='DarkBlue ' style="font-size:25px"><b>Mathematical formulation of the problem:

There are n∈ N order proposals, each with $v_i$ stocks (denoted by s) and a total cost of $w_i, 1 \leq i \leq n$. Without restriction, each combination $\left(v_i, w_i\right)$ needs to come from one account, otherwise this order would have to be split into two.

$$
\text { Order }=\left[\begin{array}{lll}
v_1 & \cdots & v_n \\
w_1 & \cdots & w_n
\end{array}\right]
$$



For a given number m∈ N of investors (accounts) we define:


$$
x_{i, j}=\left\{\begin{array}{c}
1, \text { if Order } i \text { is from account } j \\
0, \text { else. }
\end{array}\right.
$$


Let us visualize these variables with a matrix. The columns are investors (accounts), and the rows are the order proposals.

$$
\begin{array}{ccccc}
\backslash & I_1 & I_2 & \cdots & I_m \\
O_1 & x_{1,1} & x_{1,2} & \cdots & x_{1, m} \\
O_2 & x_{2,1} & x_{2,2} & \cdots & x_{2, m} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
O_n & x_{n, 1} & x_{n, 2} & \cdots & x_{n, m}
\end{array}
$$


A logical split can now be interpreted as the column sum of this matrix.

Splitting of order proposals is allowed if:

Each investor has less than  W=33s:

$$
\sum_{i=1}^n x_{i, j} \cdot v_i \leq W, \quad \forall 1 \leq j \leq m
$$


Each investor spend less has less than  T=$ 23K:

$$
\sum_{i=1}^n x_{i, j} \cdot w_i \leq I, \quad \forall 1 \leq j \leq m
$$

Each order is from exactly one account:

$$
\sum_{j=1}^m x_{i, j}=1, \quad \forall 1 \leq i \leq n
$$

Not every permissible division provides a satisfactory result. If every account is half used, that is permissible, but certainly not optimal. Therefore, **we always want the money from the last account to be as least spend as possibl**e. Remember, our goal is to have as least investors employed as possible.

In [78]:
# Read in the data from the CSV file using Pandas
df = pd.read_csv("Stocks.csv")

# Define the prices for each asset
prices = dict(zip(df["Asset Name"],df["Price"]))

# Define the maximum investment amount per investor
max_investment = float(23000)

# Define the maximum number of stocks per investor
max_stocks_per_investor = 33.0

# Define the minimum number of stocks per asset
min_stocks_per_asset = 33.0

# Define the list of investors
investors = [f"Investor {i+1}" for i in range(int(math.ceil(max_investment / min(prices.values()))))]

# Define the list of assets
assets = list(prices.keys())

# Create the binary decision variables
buy = LpVariable.dicts("Buy",[(i,j) for i in investors for j in assets], cat = "Binary")

# Create a LP problem instance
prob = LpProblem("Stocks", LpMinimize)

# Set the objective function to minimize the number of investors needed to purchase all assets
prob += lpSum([buy[(i,j)] for i in investors for j in assets]), "Minimiye Number of Investors"

# Add the constraints: each investor should invest no more than $23,000, and each investor can own at most 33 stocks
for i in investors:
  prob += lpSum([prices[j]*buy[(i,j)] for j in assets]) <= max_investment, (f"{i} can invest at most ${max_investment}")
  prob += lpSum([buy[(i,j)] for j in assets]) <= max_stocks_per_investor, f"{i} can own at most {max_stocks_per_investor}"

# Add the constraint: each asset should have at least 33 stocks purchased
for j in assets:
  prob += lpSum([buy[(i, j)] for i in investors]) >= min_stocks_per_asset, f"{j} should have at least {min_stocks_per_asset} stocks purchased"

# Solve the problem
prob.solve()

# Print the status of the problem
print("Status:", LpStatus[prob.status])

# Check if the problem has a feasible solution
if prob.status == LpStatusOptimal:
  # Print the minimum number of investors required to purchase 33 stocks of all assets
  print("Minimum number of investors needed to purchase 33 stocks of all assets:", int(prob.objective.value()))
else:
  print("No feasible solution found.")

Status: Infeasible
No feasible solution found.


This model gives me an answer that is not possible to got minimum number of investor with this constraints.